In [1]:
import sys
import pandas as pd
import os
sys.path.append(os.path.abspath('..'))
from pipeline_test_lightgbm import get_train_val_test_scaled
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, r2_score

X_train, X_val, X_test, y_train, y_val, y_test, cat_cols = get_train_val_test_scaled()

c:\Users\jmlar\OneDrive\Documentos\GitHub\ML_group_project\pipeline_test_lightgbm.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  db['odds_hw'] = db[home_win_cols].mean(axis=1)
c:\Users\jmlar\OneDrive\Documentos\GitHub\ML_group_project\pipeline_test_lightgbm.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  db['odds_d']  = db[draw_cols].mean(axis=1)
c:\Users\jmlar\OneDrive\Documentos\GitHub\ML_group_project\pipeline_test_lightgbm.py:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of ca

In [2]:
# Train models separately for home and away goals
model_home = LGBMRegressor(random_state=42)
model_away = LGBMRegressor(random_state=42)

model_home.fit(X_train, y_train['FTHG'], categorical_feature=cat_cols)
model_away.fit(X_train, y_train['FTAG'], categorical_feature=cat_cols)

c:\Users\jmlar\AppData\Local\Programs\Python\Python313\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\jmlar\AppData\Local\Programs\Python\Python313\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
        "wmic CPU Get NumberOfCores /Format:csv".split(),
        capture_output=True,
        text=True,
    )
  File "c:\Users\jmlar\AppData\Local\Programs\Python\Python313\Lib\subprocess.py", line 554, in run
    with Popen(*popenargs, **kwargs) as process:
         ~~~~~^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\jmlar\AppData\Local\Programs\Python\Python313\Lib\subprocess.py", line 1036

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005677 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6970
[LightGBM] [Info] Number of data points in the train set: 3003, number of used features: 90
[LightGBM] [Info] Start training from score 1.555778
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003377 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6970
[LightGBM] [Info] Number of data points in the train set: 3003, number of used features: 90
[LightGBM] [Info] Start training from score 1.297036


LGBMRegressor(random_state=42)

In [3]:
# Predictions
y_val_pred_home = model_home.predict(X_val)
y_val_pred_away = model_away.predict(X_val)
y_test_pred_home = model_home.predict(X_test)
y_test_pred_away = model_away.predict(X_test)

# Metrics function
def print_metrics(y_true_home, y_pred_home, y_true_away, y_pred_away, prefix=""):
    print(f"{prefix} Home Goals - MSE: {mean_squared_error(y_true_home, y_pred_home):.4f}, R²: {r2_score(y_true_home, y_pred_home):.4f}")
    print(f"{prefix} Away Goals - MSE: {mean_squared_error(y_true_away, y_pred_away):.4f}, R²: {r2_score(y_true_away, y_pred_away):.4f}")

print_metrics(y_val['FTHG'], y_val_pred_home, y_val['FTAG'], y_val_pred_away, prefix="Validation Metrics:")
print_metrics(y_test['FTHG'], y_test_pred_home, y_test['FTAG'], y_test_pred_away, prefix="Test Metrics:")

Validation Metrics: Home Goals - MSE: 0.2813, R²: -0.1720
Validation Metrics: Away Goals - MSE: 0.7275, R²: 0.2422
Test Metrics: Home Goals - MSE: 0.7454, R²: 0.5341
Test Metrics: Away Goals - MSE: 0.3324, R²: 0.4806


In [4]:
from lightgbm import LGBMRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer, mean_squared_error, r2_score

# Define model
model_home = LGBMRegressor(random_state=42)

# Define parameter grid (you can add more)
param_dist = {
    'num_leaves': [31, 50, 70, 100],
    'max_depth': [-1, 10, 20, 30],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'n_estimators': [100, 300, 500],
    'min_child_samples': [10, 20, 30],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}

# Use negative MSE as scoring (lower is better)
def mse_scorer(y_true, y_pred):
    return -mean_squared_error(y_true, y_pred)

scorer = make_scorer(mse_scorer)

# Setup randomized search
random_search = RandomizedSearchCV(
    estimator=model_home,
    param_distributions=param_dist,
    n_iter=30,           # number of parameter settings to try
    scoring=scorer,
    cv=3,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

# Fit on training data
random_search.fit(X_train, y_train['FTHG'], categorical_feature=cat_cols)

print("Best params:", random_search.best_params_)
print("Best score (neg MSE):", random_search.best_score_)

# Use best model for predictions
best_model_home = random_search.best_estimator_


Fitting 3 folds for each of 30 candidates, totalling 90 fits
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004286 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6970
[LightGBM] [Info] Number of data points in the train set: 3003, number of used features: 90
[LightGBM] [Info] Start training from score 1.555778
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Best params: {'subsample': 0.8, 'num_leaves': 70, 'n_estimators': 500, 'min_child_samples': 30, 'max_depth': 20, 'learning_rate': 0.01, 'colsample_bytree': 0.8}
Best score (neg MSE): -0.7236457706587675


In [5]:
from lightgbm import LGBMRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer, mean_squared_error

# Define model
model_away = LGBMRegressor(random_state=42)

# Parameter distribution (same as for home, adjust if you want)
param_dist = {
    'num_leaves': [31, 50, 70, 100],
    'max_depth': [-1, 10, 20, 30],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'n_estimators': [100, 300, 500],
    'min_child_samples': [10, 20, 30],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}

# Negative MSE scorer (to minimize MSE)
def mse_scorer(y_true, y_pred):
    return -mean_squared_error(y_true, y_pred)

scorer = make_scorer(mse_scorer)

# Randomized search setup
random_search_away = RandomizedSearchCV(
    estimator=model_away,
    param_distributions=param_dist,
    n_iter=30,
    scoring=scorer,
    cv=3,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

# Fit on training data for away goals
random_search_away.fit(X_train, y_train['FTAG'], categorical_feature=cat_cols)

print("Best params (away):", random_search_away.best_params_)
print("Best score (neg MSE, away):", random_search_away.best_score_)

# Best model for away goals
best_model_away = random_search_away.best_estimator_


Fitting 3 folds for each of 30 candidates, totalling 90 fits
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003987 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6970
[LightGBM] [Info] Number of data points in the train set: 3003, number of used features: 90
[LightGBM] [Info] Start training from score 1.297036
Best params (away): {'subsample': 0.8, 'num_leaves': 50, 'n_estimators': 500, 'min_child_samples': 30, 'max_depth': 20, 'learning_rate': 0.01, 'colsample_bytree': 1.0}
Best score (neg MSE, away): -0.644452901838832


In [ ]:
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Best params from tuning
best_params_home = {
    'subsample': 0.8,
    'num_leaves': 50, #Different from the best params as best results came from 50 num_leaves
    'n_estimators': 500,
    'min_child_samples': 30,
    'max_depth': 20,
    'learning_rate': 0.01,
    'colsample_bytree': 0.8,
    'random_state': 42
}

best_params_away = {
    'subsample': 0.8,
    'num_leaves': 60, #Different from the best params as best results came from 60 num_leaves
    'n_estimators': 500,
    'min_child_samples': 30,
    'max_depth': 20,
    'learning_rate': 0.01,
    'colsample_bytree': 1.0,
    'random_state': 42
}

# Initialize models
model_home = LGBMRegressor(**best_params_home)
model_away = LGBMRegressor(**best_params_away)

# Fit models
model_home.fit(X_train, y_train['FTHG'], categorical_feature=cat_cols)
model_away.fit(X_train, y_train['FTAG'], categorical_feature=cat_cols)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003860 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6970
[LightGBM] [Info] Number of data points in the train set: 3003, number of used features: 90
[LightGBM] [Info] Start training from score 1.555778
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002794 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6970
[LightGBM] [Info] Number of data points in the train set: 3003, number of used features: 90
[LightGBM] [Info] Start training from score 1.297036


LGBMRegressor(learning_rate=0.01, max_depth=20, min_child_samples=30,
              n_estimators=500, num_leaves=60, random_state=42, subsample=0.8)

In [31]:
# Predictions
y_val_pred_home = model_home.predict(X_val)
y_val_pred_away = model_away.predict(X_val)
y_test_pred_home = model_home.predict(X_test)
y_test_pred_away = model_away.predict(X_test)

# Metrics function
def print_metrics(y_true, y_pred, label):
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    print(f"{label} - MSE: {mse:.4f}, R²: {r2:.4f}")

print("Validation Metrics:")
print_metrics(y_val['FTHG'], y_val_pred_home, "Home Goals")
print_metrics(y_val['FTAG'], y_val_pred_away, "Away Goals")

print("\nTest Metrics:")
print_metrics(y_test['FTHG'], y_test_pred_home, "Home Goals")
print_metrics(y_test['FTAG'], y_test_pred_away, "Away Goals")

# Sample predictions dataframe
sample_df = pd.DataFrame({
    'Actual_FTHG': y_test['FTHG'],
    'Pred_FTHG': y_test_pred_home,
    'Actual_FTAG': y_test['FTAG'],
    'Pred_FTAG': y_test_pred_away
}).head(10)

print("\nSample Predictions on Test Set:")
print(sample_df)

Validation Metrics:
Home Goals - MSE: 0.2270, R²: 0.0543
Away Goals - MSE: 0.6844, R²: 0.2871

Test Metrics:
Home Goals - MSE: 0.5375, R²: 0.6640
Away Goals - MSE: 0.3694, R²: 0.4227

Sample Predictions on Test Set:
      Actual_FTHG  Pred_FTHG  Actual_FTAG  Pred_FTAG
3008            4   2.870336            3   1.893918
3009            1   1.295504            1   1.049076
3010            1   1.396841            1   1.219631
3011            3   2.042087            1   0.263313
3012            1   0.500884            1   0.825432
